## sentiment analysis - TFIDF feature weighting with RNN classification

`Term Weighting = TFIDF`

# Libraries

In [1]:
import pandas as pd
import itertools
import matplotlib.pyplot as plt

import numpy as np
import tensorflow as tf

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer #Count Vector Space Model
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn import metrics #Matrix Builder
from sklearn.metrics import accuracy_score  
from sklearn.model_selection import KFold #Import KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from scipy.sparse import csr_matrix

from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout

from keras import Sequential
from keras.models import load_model

from sklearn.model_selection import KFold


# Load Dataset

In [2]:
# # 5000 dataset
# # !gdown --id 1b4hiK6l4WM7EOHF61xmuwqdooT4PNXrW

# #import dataset label 400 of 5000
# !gdown --id 1FWZKEKG-VzZYWC4xGtfsNWicOY3LQZT5

_`preprocessed dataset`_

In [21]:
df = pd.read_csv('D:\kuliah\THE ONLY TA THINGS\DATA\data clean\cleaned_15000_data_sample.csv')
df

,content,score,at,label,cleansing,case_folding,no_unwanted,tokenize,normalization,stopwords,stemming,clean
0,Keren,5,2023-04-14 19:21:43,1,Keren,keren,keren,['keren'],['keren'],['keren'],['keren'],keren
1,Woy tiktok menangis gw event mlbb creator base...,5,2023-04-07 23:01:39,1,Woy tiktok menangis gw event mlbb creator base...,woy tiktok menangis gw event mlbb creator base...,woy tiktok menangis gw event mlbb creator base...,"['woy', 'tiktok', 'menangis', 'gw', 'event', '...","['oi', 'tiktok', 'menangis', 'gue', 'event', '...","['tiktok', 'menangis', 'gue', 'event', 'mlbb',...","['tiktok', 'menang', 'gue', 'event', 'mlbb', '...",tiktok menang gue event mlbb creator base gue ...
2,Halo disini saya ingin menyampaikan kepada and...,4,2023-04-07 20:13:21,1,Halo disini saya ingin menyampaikan kepada and...,halo disini saya ingin menyampaikan kepada and...,halo disini saya ingin menyampaikan kepada and...,"['halo', 'disini', 'saya', 'ingin', 'menyampai...","['halo', 'di sini', 'saya', 'ingin', 'menyampa...","['di sini', 'membuka', 'komen', 'lag', 'koneks...","['di sini', 'buka', 'komen', 'lag', 'koneksi',...",di sini buka komen lag koneksi internet bagus ...
3,Jangan mau main tiktokla paket kuota cpt hbis ...,2,2023-04-11 17:06:47,-1,Jangan mau main tiktokla paket kuota cpt hbis ...,jangan mau main tiktokla paket kuota cpt hbis ...,jangan mau main tiktokla paket kuota cpt hbis ...,"['jangan', 'mau', 'main', 'tiktokla', 'paket',...","['jangan', 'mau', 'main', 'tiktokla', 'paket',...","['main', 'tiktokla', 'paket', 'kuota', 'cepat'...","['main', 'tiktokla', 'paket', 'kuota', 'cepat'...",main tiktokla paket kuota cepat habis potong k...
4,Sangat menghibur asyik dan seru,5,2023-04-13 21:08:45,1,Sangat menghibur asyik dan seru,sangat menghibur asyik dan seru,sangat menghibur asyik dan seru,"['sangat', 'menghibur', 'asyik', 'dan', 'seru']","['sangat', 'menghibur', 'asyik', 'dan', 'seru']","['menghibur', 'seru']","['hibur', 'seru']",hibur seru
...,...,...,...,...,...,...,...,...,...,...,...,...
7833,Bagus,5,2023-04-12 07:08:12,1,Bagus,bagus,bagus,['bagus'],['bagus'],['bagus'],['bagus'],bagus
7834,Sangat dramatis dan seru,5,2023-04-07 04:51:48,1,Sangat dramatis dan seru,sangat dramatis dan seru,sangat dramatis dan seru,"['sangat', 'dramatis', 'dan', 'seru']","['sangat', 'dramatis', 'dan', 'seru']","['dramatis', 'seru']","['dramatis', 'seru']",dramatis seru
7835,Baik buat mata +18,5,2023-04-09 10:05:38,1,Baik buat mata,baik buat mata,baik buat mata,"['baik', 'buat', 'mata']","['baik', 'buat', 'mata']",[],[],NaN
7836,min ini gimana gabisa masuk min 😦😦,1,2023-04-10 18:09:14,-1,min ini gimana gabisa masuk min,min ini gimana gabisa masuk min,min ini gimana gabisa masuk min,"['min', 'ini', 'gimana', 'gabisa', 'masuk', 'm...","['min', 'ini', 'bagaimana', 'enggak bisa', 'ma...",['enggak bisa'],['enggak bisa'],enggak bisa


In [22]:
df = df.dropna()
df = df.reset_index(drop=True)
df = df[df.stemming != '[]']

In [23]:
df.isnull().sum()

content          0
score            0
at               0
label            0
cleansing        0
case_folding     0
no_unwanted      0
tokenize         0
normalization    0
stopwords        0
stemming         0
clean            0
dtype: int64

_`Seperate label to its own array`_

In [6]:
label = []
for data in df['label']:
    label.append(data)
kolom = label.pop

print(label)

[1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, -1, 1, 1, 1, 1, 1, -1, 0, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 0, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, -1, 1, 1, 1, -1, 1, -1, 1, 1, -1, 1, 1, -1, 1, 1, 1, 1, -1, 1, -1, 1, 1, -1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, 1, 0, 1, 1, 1, 1, -1, 1, 1, 1, 0, -1, 1, 1, 1, 1, -1, 1, 1, 0, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1, -1, 1, 1, 1, -1, 1, 0, 1, 1, -1, 1, 1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, 1, 1, 0, 1, 1, 1, 1, -1, 0, 1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1

In [7]:
# # Split the data into features and labels
# df['review_clean'] = df['clean'].astype(str)
# review = df['review_clean'].values

# label = pd.get_dummies(df['label']).values.astype(int)

_`total label value`_

In [27]:
df['label'].value_counts()

label
 1    5738
-1    1118
 0     329
Name: count, dtype: int64

In [ ]:
# # Tokenize the tweets
# tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=5000, oov_token='<OOV>')
# tokenizer.fit_on_texts(review)
# tokenized_review = tokenizer.texts_to_sequences(review)

In [ ]:
# # Pad the sequences
# max_length = max([len(i) for i in tokenized_review])
# tokenized_review_in_max_length = tf.keras.preprocessing.sequence.pad_sequences(tokenized_review, maxlen=max_length, padding='post')

# TFIDF for feature weighting

#### split data test data train

In [28]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['stemming'], df['label'], test_size=0.2, random_state=42)

In [29]:
# X = df["stemming"]
# y = df["label"] #data target

In [30]:
tfidf_vect = TfidfVectorizer(use_idf = True ,max_features = 5000)
x = tfidf_vect.fit(X_train)
TFIDF_train = x.transform(X_train)
TFIDF_train = TFIDF_train.toarray()

In [11]:
tfidf_vect = TfidfVectorizer(use_idf = True ,max_features = 5000)
x = tfidf_vect.fit(X_test)
TFIDF_test = x.transform(X_test)
TFIDF_test = TFIDF_test.toarray()

### y_train y_test

In [ ]:
# tfidf_vect = TfidfVectorizer(use_idf = True ,max_features = 5000)
# y = tfidf_vect.fit(y_train)
# y_train = y.transform(y_train)
# y_train = TFIDF_test.toarray()

In [ ]:
# tfidf_vect = TfidfVectorizer(use_idf = True ,max_features = 5000)
# y = tfidf_vect.fit(y_test)
# y_test = y.transform(y_test)
# y_test = TFIDF_test.toarray()

In [37]:
label = []
for data in df['label']:
    label.append(data)
kolom = label.pop

print(label)

[1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, -1, 1, 1, 1, 1, 1, -1, 0, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 0, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, -1, 1, 1, 1, -1, 1, -1, 1, 1, -1, 1, 1, -1, 1, 1, 1, 1, -1, 1, -1, 1, 1, -1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, 1, 0, 1, 1, 1, 1, -1, 1, 1, 1, 0, -1, 1, 1, 1, 1, -1, 1, 1, 0, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1, -1, 1, 1, 1, -1, 1, 0, 1, 1, -1, 1, 1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, 1, 1, 0, 1, 1, 1, 1, -1, 0, 1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1

In [31]:
from keras_preprocessing.sequence import pad_sequences
# maxlen = 121 # set the maximum sequence length to 121
maxlen = max([len(i) for i in TFIDF_train])
TFIDF_train = pad_sequences(TFIDF_train, maxlen=maxlen)
TFIDF_test = pad_sequences(TFIDF_test, maxlen=maxlen)

print("TFIDF_train shape", TFIDF_train.shape)
print("TFIDF_test shape", TFIDF_test.shape)
print("X_train shape", X_train.shape)
print("X_test shape", X_test.shape)

TFIDF_train shape (5748, 3560)
TFIDF_test shape (1437, 3560)
X_train shape (5748,)
X_test shape (1437,)


## Define RNN Model

In [38]:
# Build RNN model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=TFIDF_train.shape[1], output_dim=64),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1, activation="linear")
])

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


# Train model
history = model.fit(TFIDF_train, y_train, epochs=10, batch_size=32)

# Evaluate model
y_pred = model.predict(TFIDF_test)
y_pred = np.where(y_pred > 0.5, 1, np.where(y_pred < -0.5, -1, 0))
accuracy = np.mean(y_pred == y_test.argmax(axis=1))
print("Test accuracy:", accuracy)

ValueError: Data cardinality is ambiguous:
  x sizes: 5748
  y sizes: 3832
Make sure all arrays contain the same number of samples.

In [39]:
# Define the RNN model
def build_model(input_dim):
    model = Sequential()
    model.add(LSTM(units=128, input_shape=(input_dim, 1)))
    model.add(Dropout(0.5))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

## implement w/ k-fold

In [40]:
# set up k-fold cross validation
num_folds = 3
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

In [41]:
# train and evaluate model
fold = 0
scores = []
for train_index, test_index in kf.split(TFIDF_train):
    fold += 1
    print('\n')
    print(f'Fold {fold}/{num_folds}\n')

    # split data into train and test sets
    X_train, X_test = TFIDF_train[train_index], TFIDF_train[test_index]
    y_train, y_test = np.array(label)[train_index], np.array(label)[test_index]


    model = build_model(input_dim=TFIDF_train.shape[1])
    model.fit(np.expand_dims(X_train, axis=2), y_train, batch_size=64, epochs=5, validation_data=(X_test, y_test))

    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    scores.append(accuracy)

    print('Folds : %d | Cross-validation accuracy : %.4f | Max, Min : %.4f, %.4f' 
            % (fold, np.mean(scores), max(scores), min(scores)))
    print("\n")





Fold 1/3

Epoch 1/5
 4/60 [=>............................] - ETA: 26:50 - loss: 0.6926 - accuracy: 0.5742

In [ ]:
# evaluate model
y_pred = (model.predict(X_test) > 0.5).astype("int32")

# y_pred = model.predict(X_test).astype('int32')
# y_pred = np.where(y_pred > 0.5, 1, np.where(y_pred < -0.5, -1, 0))
# y_pred = y_pred.argmax(axis=-1)

# y_true = np.argmax(y_test, axis=-1)
# y_pred = np.argmax(y_pred, axis=1)

# y_pred_class = np.argmax(y_pred, axis=-1)
# print("Class counts:", np.bincount(y_pred_class))


print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=4))

In [ ]:
# Train and evaluate the RNN model using k-fold cross-validation
folds = range(2,3)

for k in folds:
    scores = []
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    for train_index, val_index in kf.split(TFIDF_train):
        X_train_fold, X_val_fold = TFIDF_train[train_index], TFIDF_train[val_index]
        y_train = np.array(label)
        y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]
        model = build_model(input_dim=TFIDF_train.shape[1])
        model.fit(np.expand_dims(X_train_fold, axis=2), y_train_fold, batch_size=64, epochs=5, validation_data=(X_val_fold, y_val_fold))
        
        loss, accuracy = model.evaluate(X_val_fold, y_val_fold, verbose=0)
        scores.append(accuracy)
        
    print('Folds : %d | Cross-validation accuracy : %.3f | Max, Min : %.3f, %.3f' 
            % (k, np.mean(scores), max(scores), min(scores)))
    print("\n")

## Evaluate the RNN model on the test set

In [ ]:
loss, accuracy = model.evaluate(X_val_fold, y_val_fold, verbose=0)
print('Test accuracy:', accuracy)

In [ ]:
y_pred_test = np.round(model.predict(np.expand_dims(X_test, axis=2)))
# y_pred_test = model.predict(np.expand_dims(TFIDF_test, axis=2))
print('Confusion matrix:')
conf_matrix = confusion_matrix(y_test, y_pred_test)
print(conf_matrix)
class_names = ['-1', '0', '1']
print('Classification report:')
print(classification_report(y_test, y_pred_test, target_names=class_names))



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Plot the confusion matrix
plt.figure(figsize = (5, 5))
ax = sns.heatmap(conf_matrix, cmap = 'Blues',
                    linecolor = 'white',
                    linewidth = 1,
                    annot = True,
                    fmt = '',
                    xticklabels = ['negative', 'neutral', 'positive'],
                    yticklabels = ['negative', 'neutral', 'positive'])
ax.set_title("Confusion Matrix for TFRF with RNN\n")
ax.set_xlabel("\nPredicted Values")
ax.set_ylabel("\nActual Values")
plt.show()

In [ ]:
#K-Fold Cross Validation will iterate k times
kFoldCrossValidation = KFold(n_splits=2, random_state=0, shuffle = True)

actual_classes = np.empty([0], dtype=int)
predicted_classes = np.empty([0], dtype=int)
    
for train, test in kFoldCrossValidation.split(TFIDF_train):
    #Initiate Train and Test Data then transform to TFIDF value. Then copy to new Train and Test variables. 
    trainData, testData = TFIDF_train[train], TFIDF_train[test]
    label = np.array(label)
    trainData2, testData2 = label[train], label[test]
    
    model = build_model(input_dim=trainData.shape[1])
    model.fit(np.expand_dims(trainData, axis=2), trainData2, batch_size=64, epochs=5, verbose=0)
    loss, accuracy = model.evaluate(np.expand_dims(testData, axis=2), testData2, verbose=0)
    
    actual_classes = np.append(actual_classes, label[test])
    predicted_classes = np.append(predicted_classes, model.predict(testData))
   
    scores.append(accuracy)

conf_matrix = metrics.confusion_matrix(actual_classes, predicted_classes)
rnn_accuracy = metrics.accuracy_score(actual_classes, predicted_classes)
precision = metrics.precision_score(actual_classes, predicted_classes, average='macro')
recall = metrics.recall_score(actual_classes, predicted_classes, average='macro')
f1score = metrics.f1_score(actual_classes, predicted_classes, average='macro')

print("\nConfusion Matrix: \n", conf_matrix)
print("------------------------------------------------------------")
print(classification_report(actual_classes, predicted_classes, digits = 4,
                           target_names = ['0', '1', '2']))
print("------------------------------------------------------------")
print("\nAccuracy : %.3f, Precission : %.3f, Recall : %.3f, F1 Score : %.3f" %(rnn_accuracy, precision, recall, f1score))